# ECON408: Computational Methods in Macroeconomics

Deterministic Dynamics and Introduction to Growth Models

Jesse Perla (University of British Columbia)

# Overview

## Motivation and Materials

-   In this lecture, we will introduce (non-linear) dynamics
    -   This lets us explore stationarity and convergence
    -   We will see an additional example of a **fixed point** and
        **convergence**
-   The primary applications will be to simple models of growth, such as
    the **Solow** growth model.

## Materials

-   Adapted from QuantEcon lectures coauthored with John Stachurski and
    Thomas J. Sargent
    -   [Julia by
        Example](https://julia.quantecon.org/getting_started_julia/julia_by_example.html)
    -   [Dynamics in One
        Dimension](https://julia.quantecon.org/introduction_dynamics/scalar_dynam.html)

In [1]:
using LaTeXStrings, LinearAlgebra, Plots
default(;legendfontsize=16)

# Difference Equations

## (Nonlinear) Difference Equations

$$
x_{t+1} = g(x_t)
$$

-   A **time homogeneous first order difference equation**
    -   $g : S \to S$ for some $S \subseteq \mathbb{R}$ in the
        univariate case
    -   $S$ is called the **state space** and $x$ is called the **state
        variable**.
    -   Time homogeneity: $g$ is the same at each time $t$
    -   First order: depends on one lag (i.e., $x_{t+1}$ and $x_t$ but
        not $x_{t-1}$)

## Trajectories

-   An initial condition $x_0$ is required to solve for the sequence
    $\{x_t\}_{t=0}^\infty$
-   Given this, we can generate a **trajectory** recursively $$
    \begin{aligned}
    x_1 &= g(x_0) \\
    x_2 &= g(x_1) = g(g(x_0)) \\
    x_{t+1} &= g(x_t) = g(g(\ldots g(x_0))) \equiv g^t(x_0)
    \end{aligned}
    $$
-   If not time homogeneous, we can write $x_{t+1} = g_t(x_t)$
-   Stochastic if $x_{t+1} = g(x_t, \epsilon_{t+1})$ where
    $\epsilon_{t+1}$ is a random variable

## Linear Difference Equations

$$
x_{t+1} = a x_t + b
$$ For constants $a$ and $b$. Iterating, $$
\begin{aligned}
x_1 &= g(x_0) = a x_0 + b\\
x_2 &= g(g(x_0)) = a (a x_0 + b) + b = a^2 x_0 + a b + b\\
x_3 &= a(a^2 x_0 + a b + b) + b = a^3 x_0 + a^2 b + a b + b\\
\ldots & \\
x_t &= b \sum_{j=0}^{t-1} a^j + a^t x_0 = b \frac{1 - a^t}{1 - a} + a^t x_0
\end{aligned}
$$

## Convergence and Stability for Linear Difference Equations

-   If $|a| < 1$, then take limit
-   Then for any $x_0$ we have **global stability** $$
    \lim_{t\to\infty} x_t = \lim_{t\to\infty}g^{t-1}(x_0) = \lim_{t\to\infty}\left(b \frac{1 - a^t}{1 - a} + a^t x_0\right) =  \frac{b}{1 - a}
    $$
-   Otherwise
    -   If $a = 1$ then diverges unless $b=0$
    -   If $|a| > 1$ diverges for all $b$

## Nonlinear Difference Equations

-   We can ask the same questions for nonlinear $g(\cdot)$
-   Keep in mind the connection to the fixed points from the previous
    lecture
    -   If $g(\cdot)$ has a unique fixed point from any initial
        condition, it tells us about the dynamics
-   Connecting to **contraction mappings** etc. would help us be more
    formal, but we will stay intuitive here

# Solow Growth Model

## Models of Economic Growth

-   There are different perspectives on what makes countries grow
    -   **Malthusian models:** population growth uses all available
        resources
    -   **Capital accumulation:** more capital leads to more output,
        tradeoff of consumption today to build more capital for tomorrow
    -   **Technological progress/innovation:** new ideas lead to more
        output, so the tradeoffs are between consumption today
        vs. researching technologies for the future
-   The appropriate model depends on country and time-period
    -   Malthusian models are probably most relevant right up until
        about the time he came up with the idea

## Exogenous vs. Endogenous

-   In these, the tradeoffs are key
    -   Can be driven by some sort of decision driven by the agent’s
        themselves (e.g., government plans, consumers saving, etc.)
        endogenously
    -   Or can be exogenously chosen as not responding to policy and
        incentives
    -   You always leave some things exogenous to isolate a key force
-   Consequence of choices: what determines the longrun growth rate?
    -   Analyze with **fixed points**!
    -   In models of capital accumulation, technological limitations
        limit the longrun growth rate
    -   Often people refer to models of technological progress as
        **endogenous growth models** because the long-run growth rate is
        determined by innovation decisions rather than by limitations on
        capital accumulation